**Approach :**
1.   Query + TF-IDF + Rocchio + Gamma =0.15
2.   Query + TF-IDF + Rocchio + Gamma =0


           

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Reading csv files which contain pre-processed tokens from Documents and Queries respectively

In [0]:
import pandas as pd
df=pd.read_csv('drive/My Drive/dataset.csv')

In [0]:
import pandas as pd
dfq=pd.read_csv('drive/My Drive/dataset_query.csv')

Creating List of Document tokens

In [0]:
sent=[]
for i in range(125511):
  if(i%10000==0):
    print(i)
  tokens=df["Tokens"][i][1:len(df["Tokens"][i])-2]
  tokens_list=tokens.split(', ')
  for i in range(len(tokens_list)):
    tokens_list[i]=tokens_list[i][1:len(tokens_list[i])-1]
  sent.append(tokens_list)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [0]:
print(len(sent))

125511


In [0]:
print(sent[0])

['telegraph', 'calcutta', 'leisure', 'tuesday', 'december', '07', '2004', 'leonardo', 'dicaprio', 'one', 'moment', 'didnt', 'know', 'whether', 'laugh', 'cry', 'south', 'american', 'rainforest', 'studying', 'effect', 'mercury', 'poisoning', 'amazon', 'confronted', 'group', 'naked', 'ind', 'six', 'oscar', 'big', 'profit', 'chicago', 'sparked', 'new', 'frenzy', 'filming', 'musical', 'andrew', 'lloyd', 'webbers', 'couple', 'live', 'together', 'britain', 'without', 'getting', 'married', 'warned', 'today', 'risk', 'losing', 'home', 'po', 'may', 'come', 'crunch', 'squelch', 'creator', 'hoping', 'splash', 'french', 'fashion', 'legend', 'pierre', 'cardin', 'putting', 'much', 'empire', 'sale', 'seeking', '1', 'billion', 'couture', 'licens', 'family', 'astonishingly', 'even', 'today', 'many', 'parent', 'believe', 'bringing', 'daughter', 'security', 'gilded', 'cage', 'insulated', 'first', 'cousin', '27', 'year', 'old', 'want', 'get', 'married', 'parent', 'particularly', 'interested', 'friend', 'mi

Create list of Document sentences

In [0]:
docs2=[]
docs=sent
for st in docs:
    st=' '.join(st);
    docs2.append(st);
docs=docs2;
docs2=[]
print(docs[0])

telegraph calcutta leisure tuesday december 07 2004 leonardo dicaprio one moment didnt know whether laugh cry south american rainforest studying effect mercury poisoning amazon confronted group naked ind six oscar big profit chicago sparked new frenzy filming musical andrew lloyd webbers couple live together britain without getting married warned today risk losing home po may come crunch squelch creator hoping splash french fashion legend pierre cardin putting much empire sale seeking 1 billion couture licens family astonishingly even today many parent believe bringing daughter security gilded cage insulated first cousin 27 year old want get married parent particularly interested friend mine actor diane kruger uk premiere film national treasure london reuter


Create list of Query Tokens

In [0]:
sent_q=[]
for i in range(50):
  if(i%5==0):
    print(i)
  tokens=dfq["Tokens"][i][1:len(dfq["Tokens"][i])-1]
  tokens_list=tokens.split(', ')
  for i in range(len(tokens_list)):
    tokens_list[i]=tokens_list[i][1:len(tokens_list[i])-1]
  sent_q.append(tokens_list)

0
5
10
15
20
25
30
35
40
45


In [0]:
print(sent_q[0])

['reason', 'behind', 'protest', 'meena', 'leader', 'inclusion', 'gurjars', 'scheduled', 'tribe']


Creating List of Query Sentences

In [0]:
docs2_q=[]
docs_q=sent_q
for st in docs_q:
    st=' '.join(st);
    docs2_q.append(st);
docs_q=docs2_q;
docs2_q=[]
print(docs_q[0])

reason behind protest meena leader inclusion gurjars scheduled tribe


finding tf-idf vectorizer for docs

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

In [0]:
print(X.shape)

(125511, 376999)


finding tf-idf vectorizer for queries

In [0]:
Y = vectorizer.transform(docs_q)

In [0]:
print(Y.shape)

(50, 376999)


In [0]:
list_q=[]
for i in range(50):
  a=Y[i].toarray()
  l=list(a[0])
  list_q.append(l)
  

In [0]:
import numpy as np
array_q=np.array(list_q,dtype='float32')

In [0]:
print(array_q.shape)

(50, 376999)


In [0]:
cos_lib=np.zeros(shape=(125511,50)) # to store distances

Finding Cosine Similarity

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(Y,X)

In [0]:
print(len(cos_lib[0]))

125511


In [0]:
print(sorted(cos_lib[0],reverse=True))

In [0]:
dist=[]
for i in range(len(cos_lib[0])):
  dist.append((i,cos_lib[0][i]))

dist=sorted(dist,key=lambda x:x[1],reverse=True)

In [0]:
print(dist[0:5])

[(53871, 0.42057022480782225), (124171, 0.4137515041348493), (121321, 0.37781501957210617), (121571, 0.35941066924297405), (66898, 0.336506653547955)]


Reading csv files containing File Names and Relevant Doc names for each query

In [0]:
import pandas as pd
df3=pd.read_csv('drive/My Drive/file_names.csv')

import pandas as pd
df4=pd.read_csv('drive/My Drive/relevant_docs.csv')

Threshold taken as **0.15**

In [0]:
dist=[]
rel_docs=[] # to store relev
actual_rel_docs=[] # actual rel
rel_docs_ids=[] # to store indices of rel doc vectors
p=0;
for k in range(0,50):
  dist=[]
  a=[] # to store relev
  b=[] # actual rel
  ai=[] # to store indices of relev
  for i in range(len(cos_lib[k])):
    dist.append((i,cos_lib[k][i]))

  dist=sorted(dist,key=lambda x:x[1],reverse=True)
  print("Processing...");
  i=0;
  while(dist[i][1]>=0.15 and i<125511):
   # print(dist[i]," ",df3["File_Name"][dist[i][0]].split('\\')[-1])
    a.append(df3["File_Name"][dist[i][0]].split('\\')[-1])
    ai.append(dist[i][0])
    i=i+1;
  
  print("-----------")
  
  print("Actual Relevant docs: ")
  mdf=(df4[df4["Id"]==k+76])
  #print(mdf)
  #print(len(mdf))
  
  for i in range(len(mdf)):
    b.append(mdf["Doc_Name"][p])
    #print(mdf["Doc_Name"][p])
    p=p+1;
  rel_docs.append(a)
  actual_rel_docs.append(b) # actual rel
  rel_docs_ids.append(ai)
  print("------------------------------------")

Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
-----

In [0]:
matches=0
for i in range(50):
  print(len(actual_rel_docs[i])," ",len(set(rel_docs[i]) & set(actual_rel_docs[i])))

5   0
20   1
3   2
4   0
8   3
6   0
23   1
9   2
3   0
22   2
7   4
9   2
22   13
47   7
5   3
12   1
38   2
19   0
25   1
11   0
3   2
22   14
5   0
3   0
21   4
13   3
17   3
7   0
8   0
13   1
4   0
14   0
11   1
10   0
11   1
16   0
17   0
14   0
8   0
19   1
10   0
1   0
10   2
7   0
12   0
21   2
20   1
19   18
7   0
13   0


In [0]:
matches=0
for i in range(50):
  matches=matches+len(set(rel_docs[i]) & set(actual_rel_docs[i]))

print("Total Matches: ",matches)

Total Matches:  97


In [0]:
import numpy as np
temp=np.zeros((1,376999),dtype='float32')
for k in range(125511):
    temp=np.add(temp,(X[k].toarray())) # storing relevant vectors 

In [0]:
import pandas as pd
df5=pd.read_csv('drive/My Drive/file_names_new.csv')

Calculating Relevant Docs and Non Relevant Docs vectors for each query

In [0]:
import numpy as np
dr_rev=np.zeros((50,376999),dtype='float32')
ndr_rev=np.zeros((50,376999),dtype='float32')

for i in range(50):

  tempnr=np.zeros((1,376999),dtype='float32')
  tempr=np.zeros((1,376999),dtype='float32')
  
  
  matched_files=list(set(rel_docs[i]) & set(actual_rel_docs[i]))
  mat=len(matched_files)
  nmat=125511-mat
  print("Processing...")
  for k in range(125511):
    if(df5["File_Name"][k] in matched_files):
      tempr=np.add(tempr,(X[k].toarray())) # storing relevant vectors 
      mat=mat+1;
  
  tempnr=np.subtract(temp,tempr)

  if(mat!=0):
    dr_rev[i]=tempr/mat #finding average of all relevant doc vectors
  if(nmat!=0):
    ndr_rev[i]=tempnr/nmat;

Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...
Processing...


ROCCHIO Algorithm Implementation

In [0]:
beta=0.75
gamma=0.15
mod_q=np.zeros((50,376999),dtype='float32')
for i in range(50):
  if(len(list(set(rel_docs[i]) & set(actual_rel_docs[i])))>0): # checking no. of matches > 0 or not
    total_rel=len(list(set(rel_docs[i]) & set(actual_rel_docs[i])));
    total_non_rel=125511-total_rel
    term=beta/total_rel;
    mod_q[i]=np.add(array_q[i],term*dr_rev[i])
    mod_q[i]=np.subtract(mod_q[i],(gamma/total_non_rel)*ndr_rev[i])
  else:
    total_non_rel=125511
    mod_q[i]=array_q[i]
    mod_q[i]=np.subtract(mod_q[i],(gamma/total_non_rel)*ndr_rev[i])

finding cosine Similarity with the modified query

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib2 = cosine_similarity(mod_q,X)

In [0]:
rel_docs2=[] # to store relev
p=0;
for k in range(0,50):
  dist2=[]
  a2=[] # to store relev
  b2=[] # actual rel
  for i in range(len(cos_lib2[k])):
    dist2.append((i,cos_lib2[k][i]))

  dist2=sorted(dist2,key=lambda x:x[1],reverse=True)
  print("Processing...");
  i=0;
  while(dist2[i][1]>=0.15 and i<125511):
   # print(dist[i]," ",df3["File_Name"][dist[i][0]].split('\\')[-1])
    a2.append(df3["File_Name"][dist2[i][0]].split('\\')[-1])
    i=i+1;
  
  rel_docs2.append(a2)

  print("------------------------------------")

Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
-----------------

In [0]:
matches=0
for i in range(50):
  matches=matches+len(set(rel_docs2[i]) & set(actual_rel_docs[i]))

print("Total Matches: ",matches)

Total Matches:  104


In [0]:
print("Average Precison: ", matches/125511)
print("Average Recall: ", matches/653)

Average Precison:  0.0008286126315621738
Average Recall:  0.15926493108728942


As we can see, no. of matches increased from 97 to 104
after implementing Rocchio Algorithm


Now for Gamma=0

In [0]:
beta=0.75
gamma=0
mod_q=np.zeros((50,376999),dtype='float32')
for i in range(50):
  if(len(list(set(rel_docs[i]) & set(actual_rel_docs[i])))>0): # checking no. of matches > 0 or not
    total_rel=len(list(set(rel_docs[i]) & set(actual_rel_docs[i])));
    total_non_rel=125511-total_rel
    term=beta/total_rel;
    mod_q[i]=np.add(array_q[i],term*dr_rev[i])
    mod_q[i]=np.subtract(mod_q[i],(gamma/total_non_rel)*ndr_rev[i])
  else:
    total_non_rel=125511
    mod_q[i]=array_q[i]
    mod_q[i]=np.subtract(mod_q[i],(gamma/total_non_rel)*ndr_rev[i])

In [0]:
#finding relevant with the modified query

from sklearn.metrics.pairwise import cosine_similarity
cos_lib2 = cosine_similarity(mod_q,X)

In [0]:
rel_docs2=[] # to store relev
p=0;
for k in range(0,50):
  dist2=[]
  a2=[] # to store relev
  b2=[] # actual rel
  for i in range(len(cos_lib2[k])):
    dist2.append((i,cos_lib2[k][i]))

  dist2=sorted(dist2,key=lambda x:x[1],reverse=True)
  print("Processing...");
  i=0;
  while(dist2[i][1]>=0.15 and i<125511):
   # print(dist[i]," ",df3["File_Name"][dist[i][0]].split('\\')[-1])
    a2.append(df3["File_Name"][dist2[i][0]].split('\\')[-1])
    i=i+1;
  
  rel_docs2.append(a2)

  print("------------------------------------")

Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
-----------------

In [0]:
matches=0
for i in range(50):
  matches=matches+len(set(rel_docs2[i]) & set(actual_rel_docs[i]))

print("Total Matches: ",matches)

Total Matches:  104


In [0]:
print("Average Precison: ", matches/125511)
print("Average Recall: ", matches/653)

Average Precison:  0.0008286126315621738
Average Recall:  0.15926493108728942


We observe that no. of matches do not increase after
changing gamma to 0